In [ ]:
from random import shuffle
from scipy.stats import zipf
from numpy.random import choice

In [ ]:
# number of pick faces
no_pick_faces = 10

# zipf's law shape parameter
zipf_shape = 1.1

# either "random" or "pareto"
assignment = "pareto"

In [ ]:
def generate_product_probabilities_pareto(zipf_shape, products):
    # to get wikipedia zipf from scipy zipf, just truncate
    distribution = zipf(zipf_shape)
    freqs = [distribution.pmf(n+1) for n in range(len(products))]
    freqs = freqs / sum(freqs)
    shuffle(freqs)

    return dict(zip(products, freqs))

In [ ]:
# assignment of products to pick faces
def assign_products_to_pick_faces_random(products, pick_faces):
    # random
    pick_face_assignments = pick_faces.copy()
    shuffle(pick_face_assignments)

    product_to_pick_face = dict(zip(products, pick_face_assignments))
    pick_face_to_product = dict(zip(product_to_pick_face.values(), product_to_pick_face.keys()))
    
    return product_to_pick_face, pick_face_to_product

def assign_products_to_pick_faces_pareto(products, pick_faces, product_probabilities):
    # according to freq (akin to Andrew's pareto method)
    product_probs_for_assignment = list(product_probabilities.items())
    product_probs_for_assignment.sort(key=lambda x: -x[1])
    product_to_pick_face = dict(zip(map(lambda x: x[0], product_probs_for_assignment), pick_faces))
    pick_face_to_product = dict(zip(product_to_pick_face.values(), product_to_pick_face.keys()))
    
    return product_to_pick_face, pick_face_to_product

In [ ]:
def generate_new_order(number_of_products, product_probabilities):
    labels, weights = zip(*product_probabilities.items())
    return list(choice(labels, number_of_products, p=weights))

In [ ]:
# the pick face labels
pick_faces = ["face {}".format(n+1) for n in range(no_pick_faces)]

# there's the same number of products as pick faces
products = ["product {}".format(n+1) for n in range(no_pick_faces)]

product_probabilities = generate_product_probabilities_pareto(zipf_shape, products)
product_to_pick_face, pick_face_to_product = assign_products_to_pick_faces_pareto(products, pick_faces, product_probabilities)

pick_faces, products, product_probabilities, product_to_pick_face, pick_face_to_product

In [ ]:
generate_new_order(5, product_probabilities)